In [8]:

#导入模块
import numpy as np
 
#创建加载数据读取数据以及划分数据集的函数，返回数据特征集以及数据标签集
def loaddataset(filename):
    fp = open(filename)#（299，22）
 
    # 存放数据
    dataset = []
 
    # 存放标签
    labelset = []
    for i in fp.readlines():#按照行来进行读取，每次读取一行，一行的数据作为一个元素存放在了类别中
        a = i.strip().split()#去掉每一行数据的空格以及按照默认的分隔符进行划分
 
        # 每个数据行的最后一个是标签
        dataset.append([float(j) for j in a[:len(a) - 1]])#读取每一行中除最后一个元素的前面的元素，并且将其转换为浮点数
        labelset.append(int(float(a[-1])))#读取每一行的最后一个数据作为标签数据
    return dataset, labelset#dataset是（299,21）的列表，labelset是（299,1）的列表
 
 
# x为输入层神经元个数，y为隐层神经元个数，z输出层神经元个数
#创建的是参数初始化函数，参数有各层间的权重weight和阈值即偏置value就是b
#本例的x,y=len(dataset[0])=22，z=1
def parameter_initialization(x, y, z):
    # 隐层阈值
    value1 = np.random.randint(-5, 5, (1, y)).astype(np.float64)
    #随机生成（-5，5）之间的整数组成（1，y）的数组，然后再将其转为浮点数显示
 
    # 输出层阈值
    value2 = np.random.randint(-5, 5, (1, z)).astype(np.float64)
 
    # 输入层与隐层的连接权重
    weight1 = np.random.randint(-5, 5, (x, y)).astype(np.float64)
 
    # 隐层与输出层的连接权重
    weight2 = np.random.randint(-5, 5, (y, z)).astype(np.float64)
 
    return weight1, weight2, value1, value2
 
#创建激活函数sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
 
 
'''
weight1:输入层与隐层的连接权重
weight2:隐层与输出层的连接权重
value1:隐层阈值
value2:输出层阈值
权重和阈值的个数和神经网络的隐层层数有关，若隐层为n，则权重和阈值的个数为n+1
'''
 
#创建训练样本的函数，返回训练完成后的参数weight和value，这里的函数是经过一次迭代后的参数，即所有的样本经过一次训练后的参数
#具体参数的值可以通过设置迭代次数和允许误差来进行确定
def trainning(dataset, labelset, weight1, weight2, value1, value2):
    # x为步长
    x = 0.01#学习率
    for i in range(len(dataset)):#依次读取数据特征集中的元素，一个元素即为一个样本所含有的所有特征数据
 
        # 输入数据
        #（1,21）
        inputset = np.mat(dataset[i]).astype(np.float64)#每次输入一个样本，将样本的特征转化为矩阵，以浮点数显示
 
        # 数据标签
        #（1，1）
        outputset = np.mat(labelset[i]).astype(np.float64)#输入样本所对应的标签
 
        # 隐层输入，隐层的输入是由输入层的权重决定的，wx
        #input1：（1，21）.（21，21）=（1，21）
        input1 = np.dot(inputset, weight1).astype(np.float64)
 
        # 隐层输出，由隐层的输入和阈值以及激活函数决定的，这里的阈值也可以放在输入进行计算
        #sigmoid（（1，21）-（1，21））=（1，21）
        output2 = sigmoid(input1 - value1).astype(np.float64)
 
        # 输出层输入，由隐层的输出
        #（1，21）.（21，1）=（1，1）
        input2 = np.dot(output2, weight2).astype(np.float64)
 
        # 输出层输出，由输出层的输入和阈值以及激活函数决定的，这里的阈值也可以放在输出层输入进行计算
        # （1，1）.（1，1）=（1，1）
        output3 = sigmoid(input2 - value2).astype(np.float64)
 
 
        # 更新公式由矩阵运算表示
        #a:(1,1)
        a = np.multiply(output3, 1 - output3)#输出层激活函数求导后的式子，multiply对应元素相乘，dot矩阵运算
        #g:(1,1)
        g = np.multiply(a, outputset - output3)#outputset - output3：实际标签和预测标签差
        #weight2:(21,1),np.transpose(weight2):(1,21),b:(1,21)
        b = np.dot(g, np.transpose(weight2))
        #(1,21)
        c = np.multiply(output2, 1 - output2)#隐层输出激活函数求导后的式子，multiply对应元素相乘，dot矩阵运算
        #(1,21)
        e = np.multiply(b, c)
 
        value1_change = -x * e#（1，21）
        value2_change = -x * g#（1，1）
        weight1_change = x * np.dot(np.transpose(inputset), e)#（21，21）
        weight2_change = x * np.dot(np.transpose(output2), g)#（21，1）
 
        # 更新参数，权重与阈值的迭代公式
        value1 += value1_change
        value2 += value2_change
        weight1 += weight1_change
        weight2 += weight2_change
    return weight1, weight2, value1, value2
 
#创建测试样本数据的函数
def testing(dataset1, labelset1, weight1, weight2, value1, value2):
    # 记录预测正确的个数
    rightcount = 0
    for i in range(len(dataset1)):
        # 计算每一个样例的标签通过上面创建的神经网络模型后的预测值
        inputset = np.mat(dataset1[i]).astype(np.float64)
        outputset = np.mat(labelset1[i]).astype(np.float64)
        output2 = sigmoid(np.dot(inputset, weight1) - value1)
        output3 = sigmoid(np.dot(output2, weight2) - value2)
 
        # 确定其预测标签
        if output3 > 0.5:
            flag = 1
        else:
            flag = 0
        if labelset1[i] == flag:
            rightcount += 1
        # 输出预测结果
        print("预测为%d   实际为%d   评价为%f" % (flag, labelset1[i], output3))
    # 返回正确率
    return rightcount / len(dataset1)
 
def main():
    #读取训练样本数据并且进行样本划分
    dataset, labelset = loaddataset('./in4-1.txt')
    #读取测试样本数据并且进行样本划分
    dataset1, labelset1 = loaddataset('./test4-1.txt')
    #得到初始化的待估参数的值
    weight1, weight2, value1, value2 = parameter_initialization(len(dataset[0]), len(dataset[0]), 1)
    #迭代次数为1500次，迭代次数一般越大准确率越高，但是其运行时间也会增加
    for i in range(1500):
        #获得对所有训练样本训练迭代一次后的待估参数
        weight1, weight2, value1, value2 = trainning(dataset, labelset, weight1, weight2, value1, value2)
    #对测试样本进行测试，并且得到正确率
    rate = testing(dataset1, labelset1, weight1, weight2, value1, value2)
    print("正确率为%f" % (rate))
if __name__ == '__main__':
    main()


<ipython-input-8-8e4c362dfcfd>:43: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


预测为0   实际为0   评价为0.000001
预测为0   实际为0   评价为0.292676
预测为0   实际为0   评价为0.161372
预测为0   实际为0   评价为0.035445
预测为0   实际为0   评价为0.000314
预测为0   实际为0   评价为0.341183
预测为0   实际为0   评价为0.301865
预测为0   实际为0   评价为0.000119
预测为0   实际为0   评价为0.001507
预测为0   实际为0   评价为0.166745
预测为0   实际为0   评价为0.310565
预测为1   实际为0   评价为0.546033
预测为0   实际为0   评价为0.376019
预测为0   实际为0   评价为0.206215
预测为0   实际为0   评价为0.000000
预测为0   实际为0   评价为0.000008
预测为0   实际为0   评价为0.119589
预测为0   实际为0   评价为0.137507
预测为0   实际为0   评价为0.000000
预测为0   实际为0   评价为0.083640
预测为0   实际为0   评价为0.186762
预测为0   实际为0   评价为0.102691
预测为0   实际为0   评价为0.000438
预测为1   实际为0   评价为0.637153
预测为1   实际为0   评价为0.672187
预测为0   实际为0   评价为0.381344
预测为0   实际为0   评价为0.070747
预测为0   实际为0   评价为0.256962
预测为0   实际为0   评价为0.002184
预测为0   实际为0   评价为0.113262
预测为1   实际为0   评价为0.522536
预测为0   实际为0   评价为0.000011
预测为0   实际为0   评价为0.000000
预测为0   实际为0   评价为0.076683
预测为0   实际为0   评价为0.102511
预测为0   实际为0   评价为0.285087
预测为0   实际为0   评价为0.015520
预测为0   实际为0   评价为0.000000
预测为0   实际为0 